### Day 8

In [400]:
import pandas as pd
import numpy as np
import sys
#np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(threshold=100)


In [401]:
data = np.genfromtxt('input_files/day9_input.txt', delimiter=1, dtype=int)
print(data)

[[9 7 5 ... 6 5 4]
 [6 5 4 ... 5 4 3]
 [5 4 3 ... 4 3 2]
 ...
 [5 6 5 ... 8 8 9]
 [6 7 6 ... 9 7 9]
 [9 8 9 ... 5 6 8]]


In [402]:
data.shape[0]

100

In [403]:
def check_if_lower(x, y):
    try: right = data[x,y]<data[x,y+1] 
    except: pass
    try:left = data[x,y]<data[x,y-1]
    except: pass
    try:up =  data[x,y]<data[x-1,y]
    except: pass
    try:down =  data[x,y]<data[x+1,y]
    except: pass
    x_max = data.shape[0]-1
    y_max = data.shape[1]-1
    
    # normal case
    if (x>0) and (y>0) and (x<x_max) and (y<y_max):
        condition = up and down and left and right
    #edges
    elif (x==0) and (y>0) and (x<x_max) and (y<y_max):
        condition = down and left and right
    elif (x>0) and (y==0) and (x<x_max) and (y<y_max):
        condition = up and down and right
    elif (x>0) and (y>0) and (x==x_max) and (y<y_max):
        condition = up and left and right     
    elif (x>0) and (y>0) and (x<x_max) and (y==y_max):
        condition = up and down and left
    #corners
    elif (x==0) and (y==0):
        condition = down and right
    elif (x==0) and (y==y_max):
        condition = down and left
    elif (x==x_max) and (y==0):
        condition = up and right     
    elif (x==x_max) and (y==y_max):
        condition = up and left
    
    if condition:
        return True
    else:
        return False


In [404]:
def find_lower_points():
    lower_points = {}
    
    x_max = data.shape[0]
    y_max = data.shape[1]
    for x in range(0, x_max):
        for y in range(0, y_max):
            #print(x, y)
            if check_if_lower(x, y):
                lower_points[str(x)+"/"+str(y)] =int(data[x,y])

    return lower_points

In [405]:
lower_points = find_lower_points()#lower_points

In [406]:
risk_levels = np.array([v for k,v in lower_points.items()])+1

In [407]:
risk_levels.sum()

560

### Part 2

In [427]:
centres_basins = list(lower_points.keys())

In [409]:
def check_if_basin(x, y):
    try: 
        right = data[x,y]==data[x,y+1]-1
        right_coord = [x,y+1]
    except: pass
    try:
        left = data[x,y]==data[x,y-1]-1
        left_coord = [x,y-1]
    except: pass
    try:
        up =  data[x,y]==data[x-1,y]-1
        up_coord = [x-1,y]
    except: pass
    try:
        down =  data[x,y]==data[x+1,y]-1
        down_coord = [x+1,y]
    except: pass
    
    x_max = data.shape[0]-1
    y_max = data.shape[1]-1
    coordinates = []
    # normal case
    if (x>0) and (y>0) and (x<x_max) and (y<y_max):
        if down:
            coordinates.append(down_coord)
        if up:
            coordinates.append(up_coord)
        if left:
            coordinates.append(left_coord)        
        if right:
            coordinates.append(right_coord)
    #edges
    elif (x==0) and (y>0) and (x<x_max) and (y<y_max):
        if down:
            coordinates.append(down_coord)
        if left:
            coordinates.append(left_coord)        
        if right:
            coordinates.append(right_coord)
            
    elif (x>0) and (y==0) and (x<x_max) and (y<y_max):
        if down:
            coordinates.append(down_coord)
        if up:
            coordinates.append(up_coord)        
        if right:
            coordinates.append(right_coord)
        
    elif (x>0) and (y>0) and (x==x_max) and (y<y_max):
        if left:
            coordinates.append(left_coord)
        if up:
            coordinates.append(up_coord)        
        if right:
            coordinates.append(right_coord)
        
    elif (x>0) and (y>0) and (x<x_max) and (y==y_max):
        if left:
            coordinates.append(left_coord)
        if up:
            coordinates.append(up_coord)        
        if down:
            coordinates.append(down_coord)
        
    #corners
    elif (x==0) and (y==0):
        if right:
            coordinates.append(right_coord)        
        if down:
            coordinates.append(down_coord)
        
    elif (x==0) and (y==y_max):
        if left:
            coordinates.append(left_coord)        
        if down:
            coordinates.append(down_coord)
        
    elif (x==x_max) and (y==0):
        if up:
            coordinates.append(up_coord)        
        if right:
            coordinates.append(right_coord)
        
    elif (x==x_max) and (y==y_max):
        if up:
            coordinates.append(up_coord)        
        if left:
            coordinates.append(left_coord)
    
    return coordinates

def remove_9_cell(basin):
    new_basin = []
    for item in basin:
        if not data[item[0], item[1]]==9:
            new_basin.append(item)
    return new_basin

In [410]:
def find_basin(x_centre, y_centre):
    basin_list = [[x_centre, y_centre]]
    initial_len = len(basin_list)
    final_len = len(basin_list)+1
    
    while initial_len!=final_len:
        initial_len = len(basin_list)
        for i, cell in enumerate(basin_list):
            #print(i, cell)
            #print(remove_9_cell(check_if_basin(cell[0], cell[1])))
            basin_list+=remove_9_cell(check_if_basin(cell[0], cell[1]))
            basin_list = list(set(tuple(i) for i in basin_list))
            #print("here", basin_list)
            final_len = len(basin_list)
            #print("final len", final_len)
    return len(basin_list)

In [424]:
list_len_basins = []
for centre in centres_basins:
    coordinate_centre = [int(x) for x in centre.split("/")]
    x_centre = coordinate_centre[0]
    y_centre = coordinate_centre[1]

    #print("Checking coordinates", x_centre, y_centre, "value", data[x_centre, y_centre])
    #print("len basin", find_basin(x_centre,y_centre))
    list_len_basins.append(find_basin(x_centre,y_centre))

print(list_len_basins)

[4, 13, 1, 11, 7, 7, 1, 32, 54, 16, 1, 3, 18, 24, 27, 21, 4, 25, 7, 25, 7, 57, 42, 3, 6, 25, 33, 8, 22, 16, 9, 1, 21, 2, 4, 40, 27, 17, 4, 4, 53, 6, 61, 5, 6, 24, 2, 23, 3, 57, 4, 5, 3, 1, 6, 6, 2, 12, 17, 18, 32, 2, 4, 13, 15, 72, 69, 21, 4, 3, 7, 36, 86, 11, 73, 9, 19, 15, 25, 1, 48, 3, 56, 41, 8, 17, 30, 4, 13, 2, 3, 21, 4, 21, 69, 1, 2, 9, 20, 47, 32, 64, 2, 51, 51, 10, 8, 42, 7, 7, 12, 33, 18, 3, 7, 11, 3, 6, 56, 50, 60, 37, 14, 10, 9, 28, 63, 19, 1, 36, 5, 1, 10, 49, 12, 28, 35, 5, 39, 12, 8, 31, 18, 10, 17, 27, 19, 24, 5, 17, 18, 38, 46, 27, 17, 2, 2, 40, 15, 16, 28, 49, 5, 16, 4, 42, 32, 36, 29, 74, 18, 20, 42, 58, 4, 12, 7, 13, 10, 12, 23, 50, 18, 18, 76, 22, 71, 1, 4, 3, 15, 15, 6, 23, 12, 39, 36, 18, 17, 14, 11, 4, 1, 34, 22, 6, 35, 22, 30, 23, 37, 10, 26, 49, 64, 37, 36, 41, 10, 28, 28, 22, 18, 5, 3, 33, 17, 4, 5, 24, 22, 1, 2, 1, 18, 36, 18, 2, 8]


In [425]:
print(len(centres_basins))

239


In [419]:
list_len_basins = []
x_max = data.shape[0]
y_max = data.shape[1]
for x in range(0, x_max):
    for y in range(0, y_max):
        list_len_basins.append(find_basin(x,y))

print(list_len_basins)

[1, 1, 5, 4, 3, 1, 1, 1, 4, 1, 1, 1, 2, 3, 4, 8, 7, 6, 5, 1, 1, 1, 13, 11, 10, 9, 4, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 7, 8, 10, 11, 1, 1, 7, 4, 3, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 3, 4, 5, 1, 1, 2, 4, 5, 6, 7, 8, 4, 1, 1, 2, 3, 4, 1, 2, 9, 5, 1, 1, 1, 1, 1, 1, 1, 3, 5, 7, 9, 19, 17, 15, 4, 3, 1, 1, 1, 2, 1, 8, 6, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 3, 5, 6, 8, 11, 1, 1, 2, 6, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 12, 7, 5, 2, 1, 3, 5, 1, 5, 2, 1, 1, 1, 3, 6, 9, 12, 15, 1, 1, 1, 1, 1, 1, 14, 6, 3, 1, 1, 3, 5, 7, 4, 8, 32, 18, 11, 7, 4, 1, 1, 1, 2, 5, 8, 11, 14, 54, 1, 12, 11, 9, 4, 1, 1, 1, 1, 1, 1, 1, 1, 3, 6, 16, 8, 1, 1, 4, 1, 1, 1, 3, 4, 1, 1, 4, 8, 10, 13, 17, 1, 1, 1, 2, 1, 1, 1, 4, 1, 1, 1, 1, 1, 2, 5, 12, 6, 4, 1, 1, 5, 2, 1, 1, 1, 2, 6, 9, 8, 4, 2, 1, 1, 1, 2, 3, 4, 1, 4, 2, 1, 1, 1, 4, 10, 3, 1, 1, 1, 4, 7, 10, 2, 4, 14, 7, 6, 3, 1, 1, 1, 3, 1, 1, 15, 26, 31, 34, 1, 2, 1, 3, 1, 1, 

In [426]:
list_len_basins.sort()
top_3 = list_len_basins[-3:]
print(top_3)
print(top_3[0]*top_3[1]*top_3[2])

[74, 76, 86]
483664


In [415]:
list_len_basins

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 8,
 8,
 8,
 8,
 8,
 9,
 9,
 9,
 9,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 11,
 11,
 11,
 11,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 13,
 13,
 13,
 13,
 14,
 14,
 15,
 15,
 15,
 15,
 15,
 16,
 16,
 16,
 16,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 19,
 19,
 19,
 20,
 20,
 21,
 21,
 21,
 21,
 21,
 22,
 22,
 22,
 22,
 22,
 22,
 23,
 23,
 23,
 23,
 24,
 24,
 24,
 24,
 25,
 25,
 25,
 25,
 26,
 27,
 27,
 27,
 27,
 28,
 28,
 28,
 28,
 28,
 29,
 30,
 30,
 31,
 32,
 32,
 32,
 32,
 33,
 33,
 33,
 34,
 35,
 35,
 36,
 36,
 36,
 36,
 36,
 36,
 37,
 37,
 37,
 38,
 39,
 39,
 40,
 40,
 41,
 41,
 42,
 42,
 42,
 42,
 46,
 47,
 48,
 49,
 49,
 49,
 50,
 50,

In [423]:
with open('input_files/day9_input.txt', 'r') as f:
    grid = [list(line) for line in f.read().split('\n')]

rows, cols = len(grid), len(grid[0])
basins = []
explored = set()

def get_adjacent(r, c):
    adjacent = []
    for i, j in ((r-1, c), (r, c-1), (r, c+1), (r+1, c)):
        if 0 <= i < rows and 0 <= j < cols:
            adjacent.append((i, j))
    return adjacent

def get_basin_size(r, c):
    stack = [(r, c)]
    size = 0
    while stack:
        i, j = stack.pop()
        if (i, j) not in explored and grid[i][j] != '9':
            stack += get_adjacent(i, j)
            explored.add((i, j))
            size += 1
    return size

def AOC_day9_pt1():
    low_points = 0
    for r in range(rows):
        for c in range(cols):
            depth = grid[r][c]
            if all(depth < grid[i][j] for i, j in get_adjacent(r, c)):
                low_points += int(depth) + 1
    return low_points

def AOC_day9_pt2():
    for r in range(rows):
        for c in range(cols):
            if grid[r][c] != '9':
                basins.append(get_basin_size(r, c))
    basins.sort()
    #print(basins)
    #print(basins[-3] , basins[-2] , basins[-1])
    return basins[-3] * basins[-2] * basins[-1]


print(AOC_day9_pt1())
print(AOC_day9_pt2())

560
959136
